In [1]:
# Reload logging for the Notebook

import importlib
import logging

importlib.reload(logging)

<module 'logging' from '/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py'>

# Structured logging in Python

### Rob Van Gennip

linkedin.com/in/ravangen

# Rob Van Gennip

- Engineering Lead at Wave
- Over 3 years of using Python with Django
- Previously 4 years of C# and native Windows development
- Focus on distributed systems and tooling

# Topics

- Log Levels
- Logging Classes
- Structured Logging
- Demo 
- Questions

# Log Levels

- Debug
- Info
- Warning
- Error
- Critical

## Debug

Detailed internal state for diagnosing problems

Especially with a large amount of data or a high frequency, such as records of algorithm’s internal state changes in a for-loop

## Info

Confirmation things are working as expected

Understand what is happening in the application flow

one or two per function, perhaps related to handling requests or server state changed

Server started; user logged in

## Warning

Something unexpected happened or indicative of a problem for the near future

Something important but not an error

Disk space low

## Error

Something is wrong, not able to perform an operation

Failure to read or create a database record

## Critical

Something really bad happened, may be unable to continue running

Out of memory

Disk is full

# Logging Classes

- Loggers
- Handlers
- Filters
- Formatters

## Logger

- The interface that application code directly uses
- Named typically as a dot-separated hierarchy like `'a'`, `'a.b'` or `'a.b.c.d'`
- Retrieve an instance with `logging.getLogger(name)`
- Retrieving a logger same name returns a reference to the same logger

All loggers are descendants of the root logger

Each logger passes log messages on to its parent unless `propagate` is `False`

Logger never instantiated directly, but always through `getLogger`

In [2]:
# Configure root logger behaviour to output all log levels to stdout

import logging
import sys

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)

# https://docs.python.org/3/library/logging.html#logrecord-attributes
# %(asctime)s : Human-readable time when the LogRecord was created
# %(levelname)s : Text logging level for the message
# %(name)s : Name of the logger used to log the call
# %(message)s : The logged message

In [3]:
root_logger = logging.getLogger()  # no name provided

root_logger.info('Hello PyCon Canada!')

2017-11-11 18:24:05,247 - INFO - root - Hello PyCon Canada!


In [4]:
foo_bar_logger = logging.getLogger('foo.bar')

foo_bar_logger.info('Hello PyCon Canada!')

2017-11-11 18:24:05,264 - INFO - foo.bar - Hello PyCon Canada!


In [5]:
# Use __name__ to get the fully-qualified name of the module
# Example: api/user.py → 'api.user'
# __name__ is '__main__' in an interactive prompt, script, or standard input

auto_named_logger = logging.getLogger(__name__)  # recommended

auto_named_logger.info('Hello PyCon Canada!')

2017-11-11 18:24:05,275 - INFO - __main__ - Hello PyCon Canada!


Using `__name__` ensures no name collisions

## Handler

- Sends the log records to a destination
- `StreamHandler`: output to streams such as `stdout`, `stderr`
- `NullHandler`: no output
- `FileHandler`: output to a disk file
- `RotatingFileHandler`: rotates at max file size
- `TimedRotatingFileHandler`: interval based rotation
- `SmtpHandler`: send an email per record

`RotatingFileHandler`: old log files by append the extensions .1, .2 to the file name

## Filter

- Fine grained mechanism for determining which log records to output
- Returns a true value if the record is to be processed

In [6]:
TESTING = True


class NotInTestingFilter(logging.Filter):

    def filter(self, record):
        return not TESTING

## Formatter

- Specify how content of log records are transformed for output
- Format should at least include current time, level, name, and message

#### JSON Formatter

- Output each record as JSON object
    - Includes each format string argument as key/value
    - Merge `extra` dict into output object
- Conveniently machine parsable
- Stop writing custom parsers for syslog type records

In [7]:
! pip3 install python-json-logger

In [8]:
server_logger = logging.getLogger('server')
server_logger.propagate = False  # do not propagate messages up logger hierarchy

log_format = '%(asctime)s - %(levelname)s - %(request_id)s - %(message)s'
formatter = logging.Formatter(log_format)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(formatter)
server_logger.addHandler(handler)

data = {
    'request_id': '90fca134-4468-41dc-bb26-4d087d61857b',
    'duration': 42,  # not output, not in log_format
}

server_logger.info('User logout', extra=data)

# NOTE: request_id must always be provided to 'server' logger
# If not provided, a KeyError will be raised by the format string

2017-11-11 18:24:06,088 - INFO - 90fca134-4468-41dc-bb26-4d087d61857b - User logout


In [9]:
from pythonjsonlogger import jsonlogger

api_logger = logging.getLogger('api')
api_logger.propagate = False  # do not propagate messages up logger hierarchy

log_format = '%(asctime)s %(levelname)s %(message)s'  # only standard attributes
formatter = jsonlogger.JsonFormatter(log_format)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(formatter)
api_logger.addHandler(handler)

data = {
    # output despite not in log_format
    'request_id': '90fca134-4468-41dc-bb26-4d087d61857b',
    'duration': 42,
}

api_logger.info('User logout', extra=data)

{"asctime": "2017-11-11 18:24:06,111", "levelname": "INFO", "message": "User logout", "request_id": "90fca134-4468-41dc-bb26-4d087d61857b", "duration": 42}


# Structured Logging

- A way of recording key/value pairs
- Output is readable by humans and computers (easy to query)
- Include a human readable message with every log
- Build context around operations with `extra` parameter
- Eliminate any guesswork, let tools format and adapt to the data
- Not specific to JSON, logfmt alternative:
```
level=info msg="Stopping all fetchers"
    tag=stopping_fetchers id=ConsumerFetcherManager-1382721708341
    module=kafka.consumer.ConsumerFetcherManager
```

- Trying to figure out why an exception happened is infinitely easier if you know more details about who the user was, input parameters, etc
- When using a log management system that supports searching by these custom fields, then search becomes easy

# Demo

# Questions

Why even log?
- want to know what application is doing
- understand strange behaviour, things that are hard to debug
- collect data, like performance
